In [7]:
import pandas as pd
from myUtils import pickle_load, symb_perf_stats_vectorized, symb_perf_stats_vectorized_v1

file_close = 'df_close_clean'
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance"
path_data_dump = path_dir + "/VSCode_dump/"

df_c = pickle_load(path_data_dump, file_close)
df_c = df_c[-252::]
format1 = 'df_c({}):\n{}\n{}'.format(len(df_c), df_c.head(3), df_c.tail(3))
# print(format1)

In [8]:
df_c

,A,AA,AAL,AAON,AAP,AAPL,AAT,AAWW,AB,ABB,...,ZBH,ZBRA,ZD,ZEUS,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-13,155.295425,47.428848,17.120001,80.232155,227.416687,174.725937,34.127514,86.110001,45.640442,35.749691,...,121.568710,604.979980,108.040001,21.840000,57.970001,61.169998,31.129999,231.987442,45.820000,36.950001
2021-12-14,150.913498,50.089149,16.910000,78.294884,228.650330,173.324081,33.571350,87.410004,44.463573,35.252762,...,119.456329,585.859985,106.699997,22.000000,57.570000,62.270000,30.030001,227.704330,47.119999,35.869999
2021-12-15,153.328033,50.595406,16.990000,78.294884,232.292969,178.265396,34.041210,87.860001,44.003857,35.847130,...,116.936974,598.190002,108.779999,21.910000,59.910000,62.759998,28.570000,231.707886,47.549999,35.450001
2021-12-16,149.840378,52.203499,16.520000,76.904045,233.287918,171.266006,33.926144,89.059998,43.967079,36.431751,...,116.733490,592.039978,107.440002,22.700001,58.029999,63.470001,28.000000,232.436722,45.000000,35.250000
2021-12-17,150.456436,53.335121,16.950001,77.380905,227.103592,170.152481,34.185051,87.300003,43.268314,35.817898,...,117.101692,588.520020,108.660004,25.280001,61.060001,61.200001,27.850000,233.934311,44.779999,34.970001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-06,151.350006,48.310001,14.330000,77.430000,145.529999,142.910004,26.850000,100.550003,39.480000,30.950001,...,121.389999,254.210007,89.250000,37.230000,36.360001,47.139999,25.629999,153.050003,22.520000,24.230000
2022-12-07,153.729996,46.599998,13.550000,78.169998,146.649994,140.940002,26.980000,100.750000,39.360001,30.900000,...,124.190002,252.539993,88.029999,35.200001,35.049999,47.049999,25.000000,150.250000,22.990000,24.070000
2022-12-08,156.279999,47.279999,13.600000,79.919998,146.589996,142.649994,27.139999,100.639999,39.200001,30.820000,...,125.839996,252.820007,86.989998,34.220001,33.959999,47.380001,26.010000,153.679993,23.510000,24.350000


In [9]:
# symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
    # symb_perf_stats_vectorized(df_c)
symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
    symb_perf_stats_vectorized(df_c)    
caches_perf_stats_vect = []

In [10]:
drawdown

,A,AA,AAL,AAON,AAP,AAPL,AAT,AAWW,AB,ABB,...,ZBH,ZBRA,ZD,ZEUS,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-12-14,-0.028217,0.000000,-0.012266,-0.024146,0.000000,-0.008023,-0.016297,0.000000,-0.025786,-0.013900,...,-0.017376,-0.031604,-0.012403,0.000000,-0.006900,0.000000,-0.035336,-0.018463,0.000000,-0.029229
2021-12-15,-0.012669,0.000000,-0.007594,-0.024146,0.000000,0.000000,-0.002529,0.000000,-0.035858,0.000000,...,-0.038100,-0.011223,0.000000,-0.004091,0.000000,0.000000,-0.082236,-0.001205,0.000000,-0.040595
2021-12-16,-0.035127,0.000000,-0.035047,-0.041481,0.000000,-0.039264,-0.005901,0.000000,-0.036664,0.000000,...,-0.039774,-0.021389,-0.012318,0.000000,-0.031380,0.000000,-0.100546,0.000000,-0.053628,-0.046008
2021-12-17,-0.031160,0.000000,-0.009930,-0.035537,-0.026509,-0.045510,0.000000,-0.019762,-0.051974,-0.016849,...,-0.036745,-0.027207,-0.001103,0.000000,0.000000,-0.035765,-0.105365,0.000000,-0.058254,-0.053586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-06,-0.053212,-0.488597,-0.291296,-0.037419,-0.382607,-0.210266,-0.285318,-0.005833,-0.160842,-0.185537,...,-0.093089,-0.580740,-0.211572,-0.123175,-0.439581,-0.365630,-0.188924,-0.379445,-0.537672,-0.344781
2022-12-07,-0.038323,-0.506699,-0.329871,-0.028220,-0.377855,-0.221153,-0.281858,-0.003856,-0.163393,-0.186853,...,-0.072170,-0.583494,-0.222350,-0.170984,-0.459772,-0.366842,-0.208861,-0.390798,-0.528023,-0.349108
2022-12-08,-0.022372,-0.499500,-0.327399,-0.006464,-0.378110,-0.211703,-0.277599,-0.004944,-0.166794,-0.188958,...,-0.059843,-0.583032,-0.231537,-0.194065,-0.476572,-0.362401,-0.176899,-0.376890,-0.517348,-0.341536


In [11]:
def perf_eval_0(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbol as column name 
  '''
  # %%timeit
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized(df_close)
  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = drawdown.index[0].strftime('%Y-%m-%d')
      date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # print(f'df_perf mean:\n{df_perf[_cols].mean()}\n')
  # print(f'df_perf std:\n{df_perf[_cols].std()}')
  grp_CAGRUI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGRUI_std = df_perf['CAGR/UI'].std()
  grp_CAGRUI_mean_std = grp_CAGRUI_mean / grp_CAGRUI_std
  grp_CAGRUI_mean_std

  grp_CAGRStd_mean = df_perf['CAGR/Std'].mean()
  grp_CAGRStd_std = df_perf['CAGR/Std'].std()
  grp_CAGRStd_mean_std = grp_CAGRStd_mean / grp_CAGRStd_std
  grp_CAGRStd_mean_std

  grp_StdUI_mean = df_perf['Std/UI'].mean()
  grp_StdUI_std = df_perf['Std/UI'].std()
  grp_StdUI_mean_std = grp_StdUI_mean / grp_StdUI_std
  grp_StdUI_mean_std

  return df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std

In [12]:
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval_0(df_c)
print(f'grp_StdUI_mean_std :{grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std :{grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
df_perf.head()

grp_StdUI_mean_std :2.024446
grp_CAGRStd_mean_std :-0.188054
grp_CAGRUI_mean_std :0.119452


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,A,2021-12-13,2022-12-12,1.0,0.000223,0.182288,0.122337,0.009984,0.001221
1,AA,2021-12-13,2022-12-12,1.0,-0.034132,0.394182,0.106275,-0.814770,-0.086590
2,AAL,2021-12-13,2022-12-12,1.0,-0.170561,0.246386,0.143272,-4.831713,-0.692251
3,AAON,2021-12-13,2022-12-12,1.0,-0.008378,0.273996,0.089824,-0.340399,-0.030576
4,AAP,2021-12-13,2022-12-12,1.0,-0.358314,0.208399,0.112835,-15.237935,-1.719366


In [13]:
def perf_eval_1(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbol as column name 
  '''
  # %%timeit
  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized_v1(df_close)
  caches_perf_stats_vect = []
  for symbol in symbols:
      # date_first = drawdown.index[0].strftime('%Y-%m-%d')
      # date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      date_first = df_close.index[0].strftime('%Y-%m-%d')
      date_last = df_close.index[-1].strftime('%Y-%m-%d')

      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # print(f'df_perf mean:\n{df_perf[_cols].mean()}\n')
  # print(f'df_perf std:\n{df_perf[_cols].std()}')
  grp_CAGRUI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGRUI_std = df_perf['CAGR/UI'].std()
  grp_CAGRUI_mean_std = grp_CAGRUI_mean / grp_CAGRUI_std
  grp_CAGRUI_mean_std

  grp_CAGRStd_mean = df_perf['CAGR/Std'].mean()
  grp_CAGRStd_std = df_perf['CAGR/Std'].std()
  grp_CAGRStd_mean_std = grp_CAGRStd_mean / grp_CAGRStd_std
  grp_CAGRStd_mean_std

  grp_StdUI_mean = df_perf['Std/UI'].mean()
  grp_StdUI_std = df_perf['Std/UI'].std()
  grp_StdUI_mean_std = grp_StdUI_mean / grp_StdUI_std
  grp_StdUI_mean_std

  return df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std

In [ ]:
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval_1(df_c)
print(f'grp_StdUI_mean_std :{grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std :{grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
df_perf.head()

In [ ]:
# _cols2 = ['TSBK', 'ELF', 'AVEO', 'ASC', 'TPL', 'AXON', 'NRIM', 'AAON', 'UFPT', 'GILD', 'MOD', 'SANM', 'INSW', 'LOPE', 'AE', 'ABMD', 'RMBS', 'WNC']
_cols2 = ['AE', 'ABMD', 'RMBS', 'WNC']
df_temp = df_c[_cols2].copy()
# df_perf, grp_CAGRUI_mean_std = perf_eval(df_temp)
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_temp)
# print(f'grp_CAGRUI_mean_std :{grp_CAGRUI_mean_std:.6f}')
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

In [ ]:
_df = df_perf.loc[df_perf['symbol'].isin(_cols2)]
_means = _df[['Std/UI',	'CAGR/Std', 'CAGR/UI']].mean()
_stds = _df[['Std/UI',	'CAGR/Std', 'CAGR/UI']].std()
_mean_std = _means / _stds
# print(_means, _means[0], _means[1], _means[2])
# print(_stds, _stds[0], _stds[1], _stds[2])
print(_mean_std, _mean_std[0], _mean_std[1], _mean_std[2])
_df

# df.loc[df['points'].isin([7, 9, 12])]

In [ ]:
df_SPY = df_c[['SPY']].copy()
df_SPY
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_SPY)
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()